In [ ]:
## TODO: Install any packages that you might need
## For instance, you will need the smdebug package
INSTALL_PACKAGES = False
if INSTALL_PACKAGES:
    !pip install smdebug
    !pip install tqdm

In [ ]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os
import numpy as np
import seaborn as sns

sns.set_context("talk")

BUCKET = 'project05-capstone-vexenta'

In [ ]:
model_output_dir = 's3://project03-image-classification-vexenta/model/hp-tuning/model.tar.gz'
input_train = f"s3://{BUCKET}/data/model-input/train/"
input_test = f"s3://{BUCKET}/data/model-input/test/"

os.environ["SM_MODEL_DIR"] = model_output_dir
os.environ["SM_CHANNEL_TRAIN"] = input_train
os.environ["SM_CHANNEL_TEST"] = input_test

In [ ]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "n_estimators": CategoricalParameter(100, 150, 200, 300),
    "max_depth": CategoricalParameter([10, 15, 20, 25]),
    "min_samples_split": CategoricalParameter([10, 20, 30])
}

objective_metric_name = "cv f1-score"
objective_type = "Maximize"
metric_definitions = [{"Name": "cv f1-score", "Regex": "CV F1-score: ([0-9\\.]+)"}]

In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

role = sagemaker.get_execution_role()

In [ ]:
#TODO: Create estimators for your HPs
from sagemaker.estimator import SKLearn

estimator = SKLearn(
    entry_point="../../src/modelling/00-train.py",
    role=role,
    py_version='py3',
    framework_version="0.20.0",
    instance_count=1,
    instance_type="ml.m5.large",
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=2, 
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit(
    inputs={
        "train": input_train, 
        "validation": input_valid,
    },
    wait=True
) # TODO: Remember to include your data channels

In [ ]:
tuner.best_training_job()

In [ ]:
# TODO: Get the best estimators and the best HPs
best_estimator = tuner.best_estimator() #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

In [ ]:
hyperparameters = best_estimator.hyperparameters()

hyperparameters